### Tarea 9.6

Suponga que la medición de un cierto proceso físico da como resultado la siguiente expresión

$$
p(x) = \sin x + n(x)
$$

donde $n$ es el error experimental que podemos describir como un número aleatorio, para cada valor de $x$, tomado de una distribución gaussiana con media $0$ y desviación estándar $10^{-5}$. Queremos calcular la integral

$$
\int_0^1 dx\,p(x)\,.
$$

Use la cuadratura adaptativa para calcular esta integral. El verdadero valor de esta integral es $1-\cos(1)$. Grafique el error cometido en función de la tolerancia. ¿Se puede lograr una precisión de $10^{-7}$? ¿Qué podemos decir sobre la estabilidad de la cuadratura adaptativa?

In [97]:
import numpy as np
import matplotlib.pyplot as plt

In [160]:
class MaxIterations(Exception):
    pass

def c_adaptativa(f, a, b, tol, N=100000):
    # Variables iniciales
    approx = 0
    i = 0
    toli = [10*tol]
    ai = [a]
    hi = [(b - a)/2]
    fai = [f(a)]
    fbi = [f(b)]
    fci = [f(a + hi[i])]
    S0i = [hi[i]*(fai[i] + 4*fci[i] + fbi[i])/3]
    Li = [1]
    
        
    
    while i >= 0:
        
        fd = f(ai[i] + hi[i]/2)
        fe = f(ai[i] + 3*hi[i]/2)
        S1 = hi[i]*(fai[i] + 4*fd + fci[i])/6
        S2 = hi[i]*(fci[i] + 4*fe + fbi[i])/6
        ai_prec = ai[i]
        hi_prec = hi[i]
        fai_prec = fai[i]
        fbi_prec = fbi[i]
        fci_prec = fci[i]
        toli_prec = toli[i]
        S0i_prec = S0i[i]
        Li_prec = Li[i]
        
        i -= 1
        if abs(S1 + S2 - S0i_prec) < toli_prec:
            approx += S1 + S2
        else:
            if Li_prec >= N:
                raise MaxIterations("Alcanzado máximo número de iteraciones.")
            
            # Intervalo derecho
            i += 1
            if i >= len(ai): # A veces hay que ampliar la lista
                ai.append(ai_prec + hi_prec)
                fai.append(fci_prec)
                fci.append(fe)
                fbi.append(fbi_prec)
                hi.append(hi_prec/2)
                toli.append(toli_prec/2)
                S0i.append(S2)
                Li.append(Li_prec + 1)
            else:
                ai[i] = ai_prec + hi_prec
                fai[i] = fci_prec
                fci[i] = fe
                fbi[i] = fbi_prec
                hi[i] = hi_prec/2
                toli[i] = toli_prec/2
                S0i[i] = S2
                Li[i] = Li_prec + 1
                
            # Intervalo izquierdo
            i += 1
            if i >= len(ai):
                ai.append(ai_prec)
                fai.append(fai_prec)
                fci.append(fd)
                fbi.append(fci_prec)
                hi.append(hi[i-1])
                toli.append(toli[i-1])
                S0i.append(S1)
                Li.append(Li[i-1])
            else:
                ai[i] = ai_prec
                fai[i] = fai_prec
                fci[i] = fd
                fbi[i] = fci_prec
                hi[i] = hi[i-1]
                toli[i] = toli[i-1]
                S0i[i] = S1
                Li[i] = Li[i-1]
                
    return approx

Como la función $ p(x) = \sin x + n(x) $ posee una parte constante que es $ n(x) $ podemos separar la integral y calcularlo de manera separada. Además como  $n (x) $ al ser definido como una constante, el valor de la integral da como resultado 1.

Luego desarrolamos la otra parte de la función:

In [161]:
c_adaptativa(np.sin, 0, 1, 10**(-5))

0.45969831879846146

Por tanto el valor queda de la forma $$ 1 - 0.45969831879846146 \approx 1 - cos (1)$$



Para calcular el error en la medición usamos la definición: 
$$
\frac{|Valor \ Teórico - Valor \ obtenido|}{Valor \ Teórico}
$$

Podemos calcular el valor obtenido cambiando el valor de la tolerancia para luego graficarlo

In [171]:
error = abs((0.45969769413186028 - 0.45969831879846146)/0.45969769413186028)

Para conocer el error en porcentaje, multiplicamos el resultado anterior y obtenemos:

In [163]:
e_final = error*100

In [164]:
print('El error cometido con una tolerancia de 10^-5 es:', e_final, '%')

El error cometido con una tolerancia de 10^-5 es: 0.00013588638993446812 %


Para calcular el error con una tolerancia de $10^{-7}$ repetimos el procedimiento anterior, pero cambiando la tolerancia


In [165]:
c_adaptativa(np.sin, 0, 1, 10**(-7))

0.4596977331190459

In [166]:
V_f = [0.4596977331190459]

In [167]:
error7 = abs((0.45969831879846146 - 0.4596977331190459)/0.45969769413186028)
e_final7 = error7*100

In [168]:
print('El error cometido con una tolerancia de 10^-7 es:', e_final7, '%')

El error cometido con una tolerancia de 10^-7 es: 0.00012740534117379067 %


Luego realizamos el gráfico con el valor obtenido comparando ambos resultados